# Example: Fun with Iterative Linear Algebraic Equation (LAE) Solvers
In this example, let's explore the properties and performance of three iterative linear algebraic solvers: the Jacobi method, the Gauss-Seidel method, and the Successive Over-Relaxation (SOR) method. 

> __Learning Objectives:__
>
> By the end of this activity, you should be able to:
> * __Random System Generation__: Generate random diagonally dominant system matrices $\mathbf{A}$ and right-hand-side vectors $\mathbf{b}$ of a specified dimension. We'll use these test matrices for benchmarking studies later in the activity.
> * __Solve a random square system__: Solve the LAEs using the Jacobi, Gauss-Seidel, and SOR methods. In this task, we'll solve our system of random linear algebraic equations using the [Jacobi](https://en.wikipedia.org/wiki/Jacobi_method), [Gauss-Seidel](https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method), and [SOR](https://en.wikipedia.org/wiki/Successive_over-relaxation) methods.
> * __Benchmark__: In this task, we'll compare the runtime performance of the different iterative approaches against the built-in method implemented by [the LinearAlgebra.jl package](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/) included with Julia [using the `BenchmarkTools.jl` package.](https://github.com/JuliaCI/BenchmarkTools.jl)

This should be fun, so let's go!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> __Include:__ The [include command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up your environment:

In [3]:
include(joinpath(@__DIR__, "Include.jl"));

In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl). Check out [the documentation](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/) for more information on the functions, types, and data used in this material. 

___

## Task 1: Build random diagonally dominant matrices
In this task, we'll generate a random system matrix $\mathbf{A}$ that is diagonally dominant and a random right-hand side vector $\mathbf{b}$. 

> __Diagonal dominance__ is a matrix property where the absolute value of the diagonal element of each row is greater than the sum of the absolute values of the other elements in that row. Diagonal dominance is a sufficient (but not necessary) condition for convergence of iterative methods; however, this condition says nothing about the rate of convergence.

A diagonally dominant system matrix $\mathbf{A}$ has the feature:
$$
\begin{equation*}
\sum_{j=1,j \neq i}^{n}\lvert{a_{ij}}\rvert<\lvert{a_{ii}}\rvert\qquad\forall{i}
\end{equation*}
$$

Let's start by specifying how many rows we have in the _square_ system matrix $\mathbf{A}$ in the `number_of_rows::Int64` variable:

In [6]:
number_of_rows = 100; # increase the number of rows for larger problem size

Next, we generate a $n\times{n}$ random system matrix $\mathbf{A}$ and a $n\times{1}$ random vector $\mathbf{b}$ [using the `randn(...)` method](https://docs.julialang.org/en/v1/stdlib/Random/#Base.randn). We add some extra weight to the diagonal elements of the test system matrix $\mathbf{A}$ to ensure diagonal dominance.

In [8]:
A,b = let

    # initialize -
    ϵ = 10.0; # extra stuff that we add to diagonal elements
    A = (1/ϵ)*rand(number_of_rows, number_of_rows) .+ ϵ*diagm(rand(number_of_rows)); # Interesting!
    b = ϵ*rand(number_of_rows);

    A,b
end;

In [9]:
A

100×100 Matrix{Float64}:
 9.68155     0.0471997   0.014144    …  0.0338752    0.01299     0.0552358
 0.0984892   4.88513     0.0776257      0.091426     0.038654    0.0179228
 0.0263159   0.0188777   8.40025        0.0785771    0.0850449   0.0893039
 0.0671977   0.0313656   0.0818744      0.085893     0.0521944   0.0198186
 0.046911    0.0317125   0.0857733      0.034464     0.0520169   0.0105191
 0.0061727   0.0175856   0.0239967   …  0.0500491    0.0620497   0.0103616
 0.0496227   0.00902304  0.038538       0.0672795    0.0289991   0.0830749
 0.00683204  0.0551744   0.0365714      0.0244699    0.0956617   0.0134626
 0.0776378   0.0864163   0.0115328      0.0993366    0.0415329   0.0355475
 0.0255973   0.0610524   0.066454       0.0178836    0.0052349   0.0762256
 0.0264381   0.0522949   0.0281245   …  0.0115866    0.0402488   0.0743805
 0.0284826   0.0345233   0.00157051     0.066297     0.0362274   0.0939106
 0.0520791   0.0960884   0.0110404      0.0479334    0.095275    0.0642586


What does $\mathbf{A}$ look like?

In [11]:
A

100×100 Matrix{Float64}:
 9.68155     0.0471997   0.014144    …  0.0338752    0.01299     0.0552358
 0.0984892   4.88513     0.0776257      0.091426     0.038654    0.0179228
 0.0263159   0.0188777   8.40025        0.0785771    0.0850449   0.0893039
 0.0671977   0.0313656   0.0818744      0.085893     0.0521944   0.0198186
 0.046911    0.0317125   0.0857733      0.034464     0.0520169   0.0105191
 0.0061727   0.0175856   0.0239967   …  0.0500491    0.0620497   0.0103616
 0.0496227   0.00902304  0.038538       0.0672795    0.0289991   0.0830749
 0.00683204  0.0551744   0.0365714      0.0244699    0.0956617   0.0134626
 0.0776378   0.0864163   0.0115328      0.0993366    0.0415329   0.0355475
 0.0255973   0.0610524   0.066454       0.0178836    0.0052349   0.0762256
 0.0264381   0.0522949   0.0281245   …  0.0115866    0.0402488   0.0743805
 0.0284826   0.0345233   0.00157051     0.066297     0.0362274   0.0939106
 0.0520791   0.0960884   0.0110404      0.0479334    0.095275    0.0642586


### Check: Is the system matrix $\mathbf{A}$ strictly diagonally dominant?
Before we continue, let's verify that the randomly generated system matrix $\mathbf{A}$ is actually diagonally dominant.
> __Test:__ We'll compute the sum of the absolute values of each row (excluding the diagonal element) and compare it to the absolute value of the diagonal element in that row. If the sum of the absolute values of the non-diagonal elements is less than the absolute value of the diagonal element, then the matrix is diagonally dominant.

Is the matrix $\mathbf{A}$ strictly diagonally dominant?

In [13]:
ddcondition = let
    
    # initialize -
    ddcondition = Array{Bool,1}(undef, number_of_rows);

    # let's check each row
    for i ∈ 1:number_of_rows
        aii = abs(A[i,i]);
        σ = 0.0;
        for j ∈ 1:number_of_rows
            if (i ≠ j)
                σ += abs(A[i,j]);
            end
        end
        ddcondition[i] = (aii > σ) ? true : false; # ternary operator, nice!
    end

    ddcondition
end;

In [14]:
ddcondition

100-element Vector{Bool}:
 1
 0
 1
 0
 0
 0
 1
 0
 0
 1
 1
 1
 0
 ⋮
 1
 1
 0
 1
 1
 0
 1
 1
 1
 1
 1
 1

### Understanding the Diagonal Dominance Check

The `ddcondition` array contains boolean values indicating whether each row of our system matrix $\mathbf{A}$ satisfies the diagonal dominance condition. Each element `ddcondition[i]` is `true` if the absolute value of the diagonal element in row `i` is greater than the sum of the absolute values of all other elements in that row, and `false` otherwise.

> __Test:__ The assertion `@assert any(ddcondition)` checks that at least one row (and hopefully all rows) satisfies the diagonal dominance condition. If the condition fails, a warning is issued indicating that the matrix may not be suitable for iterative methods, as convergence is not guaranteed.

For diagonally dominant matrices, iterative methods like Jacobi, Gauss-Seidel, and SOR are guaranteed to converge, making them reliable choices for solving linear systems.

In [16]:
try
    @assert any(ddcondition)
catch
    @warn "Diagonal dominance condition not satisfied"
end

__No warnings?__ Great, let's move on to solving the system of linear algebraic equations (LAEs) using one of our iterative methods.

___

## Task 2: Let's test our iterative solvers
In this task, let's test the correctness of our iterative linear solver implementations. 

> __Idea:__ We'll solve the system of linear algebraic equations $\mathbf{A}\mathbf{x}=\mathbf{b}$ using the Jacobi, Gauss-Seidel, and SOR methods and compare the results to the solution obtained using [Julia's built-in backslash operator (`\`)](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#Base.:\\-Tuple{AbstractMatrix,%20AbstractVecOrMat}), which uses a polyalgorithm approach where the choice of a specific method depends on the problem structure and size.

First, let's use the backslash operator to obtain the Julia solution (which we will use as a reference). We'll store the reference solution in the `julia_solution::Array{Float64,1}` variable.

In [45]:
julia_solution = A\b; # Julia solution

Next, let's set up a code block to compute the solutions using the Jacobi, Gauss-Seidel, and SOR methods. We'll compare the results to the Julia solution we obtained earlier.

> __Explanation:__ In the following code block, we'll solve our linear system $\mathbf{A}\mathbf{x}=\mathbf{b}$ using one of our iterative methods. 
> 
> * The code sets up the algorithm parameters including maximum iterations, convergence tolerance, and initial guess, then calls our custom solver implementation. 
> * The solver returns a dictionary (`our_solution_archive::Dict{Int64, Vector{Float64}}`) that stores the solution vector (value) at each iteration (key), allowing us to track the convergence behavior and retrieve the final converged solution. 
> * You can switch between different iterative methods (Jacobi, Gauss-Seidel, or SOR) by changing the `algorithm` parameter.

So what do we get?

In [47]:
our_solution_archive = let

    # initialize -
    maximum_number_of_iterations = 1000;
    tolerance = 1e-12;
    algorithm = GaussSeidelMethod(); # change this to GaussSeidelMethod() or SuccessiveOverRelaxationMethod() to test other algorithms
    ω = 1.0; # relaxation factor, only used for SOR
    xₒ = 0.1*ones(number_of_rows); # initial solution guess xₒ

    # call the solve method with the appropriate parameters -
    x = VLDataScienceMachineLearningPackage.solve(A, b, xₒ; algorithm = algorithm, 
        maxiterations = maximum_number_of_iterations, 
        ϵ = tolerance, 
        ω = ω # only used for SOR
    );

    x; # return the solution
end;

In [49]:
our_solution_archive

Dict{Int64, Vector{Float64}} with 29 entries:
  5  => [-0.175732, -0.370214, 0.351694, 1.71276, -2.7905, -0.439131, 1.20944, …
  16 => [-0.17663, -0.372422, 0.35058, 1.7095, -2.80944, -0.441287, 1.20759, 4.…
  20 => [-0.17663, -0.372422, 0.35058, 1.7095, -2.80944, -0.441287, 1.20759, 4.…
  12 => [-0.17663, -0.372422, 0.35058, 1.7095, -2.80944, -0.441288, 1.20759, 4.…
  24 => [-0.17663, -0.372422, 0.35058, 1.7095, -2.80944, -0.441287, 1.20759, 4.…
  28 => [-0.17663, -0.372422, 0.35058, 1.7095, -2.80944, -0.441287, 1.20759, 4.…
  8  => [-0.176609, -0.372395, 0.350582, 1.7095, -2.80956, -0.441302, 1.20759, …
  17 => [-0.17663, -0.372422, 0.35058, 1.7095, -2.80944, -0.441287, 1.20759, 4.…
  1  => [0.105321, 0.163118, 0.596922, 2.4355, 0.238522, 0.136304, 1.59266, 6.0…
  19 => [-0.17663, -0.372422, 0.35058, 1.7095, -2.80944, -0.441287, 1.20759, 4.…
  0  => [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1  …  0.1, 0.1, 0.1, 0.…
  22 => [-0.17663, -0.372422, 0.35058, 1.7095, -2.80944, -0.441

### Check: Solution consistency
Before we proceed, let's check the difference between the solution produced by Julia and the solution we computed using our iterative solution techniques. 

Let's compute the magnitude of the residual vector between our solution and the Julia solution using a [norm function](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.norm) exported by [the `LinearAlgebra.jl` package included with the standard library](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#man-linalg).

> __Vector norm__
>
> A function $|\cdot|:\mathbb{K}^{n}\to\;[0,\infty)$ ($\mathbb{K}=\mathbb{R}$ or $\mathbb{C}$) is a **norm** if, for all $u,v\in\mathbb{K}^n$ and $\alpha\in\mathbb{K}$,
>
> 1. **Definiteness:** $|v|\ge 0$ and $|v|=0 \iff v=0$.
> 2. **Homogeneity:** $|\alpha v|=|\alpha|\times|v|$.
> 3. **Triangle inequality:** $|u+v|\le |u|+|v|$.

Here, we will use a particular norm, the p-norm:

> **Definition ($p$-norm / $\ell_p$ norm).**
> 
> Let $\mathbb{K}\in{\mathbb{R},\mathbb{C}}$ and $x=(x_1,\ldots,x_n)\in\mathbb{K}^n$. Then, for $1\le p<\infty$, the p-norm is given by:
> $$
> \|x\|_{p}=\Big(\sum_{i=1}^n |x_i|^{\,p}\Big)^{1/p}.
> $$
> and for $p=\infty$:
> $$
> \|x\|_{\infty}=\max_{1\le i\le n}|x_i|.
> $$

Let's keep it simple and use the $p = 2$ norm for our residual calculation (the default).

In [51]:
let

    # initialize -
    atol = 1e-8; # absolute tolerance
    i = keys(our_solution_archive) |> collect |> sort |> last; # get the last key (the last iteration)
    last_solution = our_solution_archive[i]; # get the last solution (converged solution)

    r = last_solution - julia_solution; 
    norm_r = norm(r) # 2-norm of the residual

    # check: our solution vs Julia solution, should be approximately equal
    @assert isapprox(norm_r, 0, atol=atol)
end

If we don't have any errors, then our iterative solver is working correctly! But how well does it perform compared to Julia's built-in solver? Let's check that question next.
___

## Task 3: Let's compare the performance of our solvers to the built-in Julia solvers
In this task, we'll benchmark our iterative solvers against Julia's built-in solvers to see how they perform on the same problem.

>__Expectation:__ We expect Julia's built-in solvers to be highly optimized and potentially __much__ faster than our iterative solvers, especially for larger problem sizes. However, our solvers may still perform competitively for smaller problems or specific cases.

Let's start with Julia's built-in solvers.

In [27]:
@benchmark $A\$b

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  47.916 μs …  2.792 ms  ┊ GC (min … max): 0.00% … 96.85%
 Time  (median):     51.333 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   53.406 μs ± 46.489 μs  ┊ GC (mean ± σ):  2.59% ±  3.57%

   █▃   ▄   ▂▂                                                 
  ▃██▃▂▆██▇███▇▅▃▂▃▄▃▂▂▃▂▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  47.9 μs         Histogram: frequency by time        69.7 μs <

 Memory estimate: 81.89 KiB, allocs estimate: 7.

Now, let's benchmark our iterative solvers against Julia's built-in solvers to see how they perform on the same problem.

In [55]:
let
    # initialize -
    maximum_number_of_iterations = 10000;
    tolerance = 1e-8;
    algorithm = SuccessiveOverRelaxationMethod(); # change this to GaussSeidelMethod() or SuccessiveOverRelaxationMethod() to test other algorithms
    ω = 0.6; # relaxation factor, only used for SOR
    xₒ = 0.1*ones(number_of_rows); # initial solution guess xₒ

     # call the solve method with the appropriate parameters -
    @benchmark VLDataScienceMachineLearningPackage.solve($A, $b, $xₒ; algorithm = $algorithm, 
        maxiterations = $maximum_number_of_iterations, 
        ϵ = $tolerance, 
        ω = $ω # only used for SOR
    );
end

BenchmarkTools.Trial: 590 samples with 1 evaluation per sample.
 Range (min … max):  157.833 μs … 96.412 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):       5.787 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):     8.475 ms ± 10.085 ms  ┊ GC (mean ± σ):  1.33% ± 4.12%

  █  ▂▁ ▁▁                                                      
  ██▇██▇██▇▆▆▅▅▅▅▅▅▃▄▃▄▃▃▃▂▃▃▃▂▃▃▂▂▃▂▂▁▂▁▁▂▁▃▂▂▂▁▁▁▂▁▁▁▂▂▂▁▁▂▂ ▃
  158 μs          Histogram: frequency by time         45.1 ms <

 Memory estimate: 771.81 KiB, allocs estimate: 317.

### What did we see?
The Julia implementation is efficient, even for larger problem sizes. The iterative solvers we implemented perform comparably (similar order of magnitude median runtime) to the built-in solvers, but there are some notable differences in memory usage and convergence behavior.

> __Differences:__ Our iterative solvers consistently use significantly more memory than the built-in solvers, particularly for larger problem sizes. Furthermore, our solvers have more allocations. This is likely due to the way we store and manipulate intermediate results (e.g., for informational purposes we keep all intermediate solutions). Occasionally, our solvers may diverge or exhibit slower convergence compared to the built-in methods.

Another data point in the ongoing story that is __buy versus build__. Unless you need some specialized behavior, it's almost always better to use established libraries and frameworks rather than building your own solutions from scratch!

___